In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
import torch
import json
from utils.inc_net import OurNet
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

/home/yxma/anaconda3/envs/cllora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_json(setting_path):
    with open(setting_path) as data_file:
        param = json.load(data_file)
    return param

In [3]:
param = load_json("exps/skin_ours.json")
param["device"] = ["cpu"]
network = OurNet(param, True)
network

This is for the BaseNet initialization.
I'm using ViT with adapters.


/home/yxma/anaconda3/envs/cllora/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/yxma/anaconda3/envs/cllora/lib/python3.10/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name vit_base_patch16_224_in21k to current vit_base_patch16_224.augreg_in21k.
  model = create_fn(


_IncompatibleKeys(missing_keys=['block_weight', 'cur_adapter.0.0.lora_A.weight', 'cur_adapter.0.0.lora_B.weight', 'cur_adapter.0.1.lora_A.weight', 'cur_adapter.0.1.lora_B.weight', 'cur_adapter.0.2.lora_A.weight', 'cur_adapter.0.2.lora_B.weight', 'cur_adapter.0.3.lora_A.weight', 'cur_adapter.0.3.lora_B.weight', 'cur_adapter.1.0.lora_A.weight', 'cur_adapter.1.0.lora_B.weight', 'cur_adapter.1.1.lora_A.weight', 'cur_adapter.1.1.lora_B.weight', 'cur_adapter.1.2.lora_A.weight', 'cur_adapter.1.2.lora_B.weight', 'cur_adapter.1.3.lora_A.weight', 'cur_adapter.1.3.lora_B.weight', 'cur_adapter.2.0.lora_A.weight', 'cur_adapter.2.0.lora_B.weight', 'cur_adapter.2.1.lora_A.weight', 'cur_adapter.2.1.lora_B.weight', 'cur_adapter.2.2.lora_A.weight', 'cur_adapter.2.2.lora_B.weight', 'cur_adapter.2.3.lora_A.weight', 'cur_adapter.2.3.lora_B.weight', 'cur_adapter.3.0.lora_A.weight', 'cur_adapter.3.0.lora_B.weight', 'cur_adapter.3.1.lora_A.weight', 'cur_adapter.3.1.lora_B.weight', 'cur_adapter.3.2.lora_A.weig

OurNet(
  (backbone): VisionTransformer(
    (old_adapter_list): ModuleList()
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention_lora(
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear

In [4]:
task_ckpt_dir = "logs/ours/SKIN/2/3/no_kd_orth_all_spec_lora"
task_best_model_ckpts = []
for task in range(9):
    task_ckpt_root = os.path.join(task_ckpt_dir,f"best_model_task_{task}.pth")
    task_best_model_ckpts.append(torch.load(task_ckpt_root)['state_dict'])

In [17]:
task_best_model_ckpts[0].keys()

odict_keys(['backbone.block_weight', 'backbone.cls_token', 'backbone.pos_embed', 'backbone.patch_embed.proj.weight', 'backbone.patch_embed.proj.bias', 'backbone.blocks.0.norm1.weight', 'backbone.blocks.0.norm1.bias', 'backbone.blocks.0.attn.q_proj.weight', 'backbone.blocks.0.attn.q_proj.bias', 'backbone.blocks.0.attn.v_proj.weight', 'backbone.blocks.0.attn.v_proj.bias', 'backbone.blocks.0.attn.k_proj.weight', 'backbone.blocks.0.attn.k_proj.bias', 'backbone.blocks.0.attn.proj.weight', 'backbone.blocks.0.attn.proj.bias', 'backbone.blocks.0.norm2.weight', 'backbone.blocks.0.norm2.bias', 'backbone.blocks.0.fc1.weight', 'backbone.blocks.0.fc1.bias', 'backbone.blocks.0.fc2.weight', 'backbone.blocks.0.fc2.bias', 'backbone.blocks.1.norm1.weight', 'backbone.blocks.1.norm1.bias', 'backbone.blocks.1.attn.q_proj.weight', 'backbone.blocks.1.attn.q_proj.bias', 'backbone.blocks.1.attn.v_proj.weight', 'backbone.blocks.1.attn.v_proj.bias', 'backbone.blocks.1.attn.k_proj.weight', 'backbone.blocks.1.attn

In [18]:
def extract_lora_parameters_by_key(state_dict):
    """从状态字典中提取lora_a和lora_b参数"""
    lora_a_params = {}
    lora_b_params = {}
    
    for key, value in state_dict.items():
        if 'lora_A' in key and 'weight' in key and 'cur_adapter' in key:
            param_name = key.split('.')[-4] + '.' + key.split('.')[-3] + '.' + key.split('.')[-2]  # 提取更具体的参数名称
            lora_a_params[param_name] = value
        elif 'lora_B' in key and 'weight' in key and 'cur_adapter' in key:
            param_name = key.split('.')[-4] + '.' + key.split('.')[-3] + '.' + key.split('.')[-2]
            lora_b_params[param_name] = value
    
    return lora_a_params, lora_b_params

def calculate_key_similarity_matrix(params_list, key_name):
    """计算特定key在所有任务间的相似度矩阵"""
    n_tasks = len(params_list)
    sim_matrix = np.zeros((n_tasks, n_tasks))
    
    for i in range(n_tasks):
        for j in range(n_tasks):
            if key_name in params_list[i] and key_name in params_list[j]:
                vec_i = params_list[i][key_name].cpu().flatten().numpy()
                vec_j = params_list[j][key_name].cpu().flatten().numpy()
                
                if np.linalg.norm(vec_i) > 0 and np.linalg.norm(vec_j) > 0:
                    sim = np.dot(vec_i, vec_j) / (np.linalg.norm(vec_i) * np.linalg.norm(vec_j))
                    sim_matrix[i, j] = sim
                else:
                    sim_matrix[i, j] = 0
            else:
                sim_matrix[i, j] = 0
    
    return sim_matrix

def visualize_key_similarity(sim_matrix, key_name, param_type, save_dir):
    """可视化单个key的相似度矩阵"""
    plt.figure(figsize=(10, 8))
    
    mask = np.triu(np.ones_like(sim_matrix, dtype=bool), k=1)
    
    sns.heatmap(sim_matrix, 
                mask=mask,
                annot=True, 
                cmap='viridis',
                center=0,
                square=True,
                cbar_kws={'shrink': 0.8},
                fmt='.3f')
    
    plt.title(f'{param_type} - {key_name} Similarity Across Tasks', fontsize=14, pad=20)
    plt.xlabel('Task Index')
    plt.ylabel('Task Index')
    plt.tight_layout()
    
    # 创建保存目录
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f'{param_type}_{key_name}_similarity.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def create_summary_heatmap(all_sim_matrices, keys, param_type, save_dir):
    """创建所有keys的汇总热图"""
    n_keys = len(keys)
    n_tasks = all_sim_matrices[0].shape[0]
    
    # 计算每个key的平均对角线值（任务内相似度）
    key_scores = []
    for i, key in enumerate(keys):
        diag_mean = np.mean(np.diag(all_sim_matrices[i]))
        key_scores.append((key, diag_mean))
    
    # 按相似度排序
    key_scores.sort(key=lambda x: x[1], reverse=True)
    
    # 创建汇总图
    fig, axes = plt.subplots(n_keys, 1, figsize=(12, 3 * n_keys))
    
    if n_keys == 1:
        axes = [axes]
    
    for idx, (key, score) in enumerate(key_scores):
        sim_matrix = all_sim_matrices[keys.index(key)]
        
        sns.heatmap(sim_matrix, 
                    ax=axes[idx],
                    annot=True, 
                    cmap='viridis',
                    center=0,
                    square=True,
                    cbar_kws={'shrink': 0.8},
                    fmt='.3f')
        
        axes[idx].set_title(f'{key} (Avg Diag: {score:.3f})')
        axes[idx].set_xlabel('Task Index')
        axes[idx].set_ylabel('Task Index')
    
    plt.tight_layout()
    save_path = os.path.join(save_dir, f'{param_type}_all_keys_summary.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

In [19]:
# 按key提取lora参数
lora_a_by_key = []
lora_b_by_key = []

for i, state_dict in enumerate(task_best_model_ckpts):
    lora_a, lora_b = extract_lora_parameters_by_key(state_dict)
    lora_a_by_key.append(lora_a)
    lora_b_by_key.append(lora_b)
    print(f"Task {i}: Found {len(lora_a)} lora_A keys and {len(lora_b)} lora_B keys")

# 获取所有唯一的key
all_lora_a_keys = set()
all_lora_b_keys = set()

for params in lora_a_by_key:
    all_lora_a_keys.update(params.keys())
for params in lora_b_by_key:
    all_lora_b_keys.update(params.keys())

print(f"Unique lora_A keys: {sorted(all_lora_a_keys)}")
print(f"Unique lora_B keys: {sorted(all_lora_b_keys)}")

# 为每个lora_A key计算相似度矩阵
lora_a_sim_matrices = {}
for key in all_lora_a_keys:
    sim_matrix = calculate_key_similarity_matrix(lora_a_by_key, key)
    lora_a_sim_matrices[key] = sim_matrix
    visualize_key_similarity(sim_matrix, key, 'lora_a', task_ckpt_dir)
    print(f"Generated similarity heatmap for lora_A key: {key}")

# 为每个lora_B key计算相似度矩阵
lora_b_sim_matrices = {}
for key in all_lora_b_keys:
    sim_matrix = calculate_key_similarity_matrix(lora_b_by_key, key)
    lora_b_sim_matrices[key] = sim_matrix
    visualize_key_similarity(sim_matrix, key, 'lora_b', task_ckpt_dir)
    print(f"Generated similarity heatmap for lora_B key: {key}")

# 创建汇总图
create_summary_heatmap(list(lora_a_sim_matrices.values()), 
                        list(lora_a_sim_matrices.keys()), 
                        'lora_a', task_ckpt_dir)

create_summary_heatmap(list(lora_b_sim_matrices.values()), 
                        list(lora_b_sim_matrices.keys()), 
                        'lora_b', task_ckpt_dir)

# 保存相似度矩阵数据
np.savez(os.path.join(task_ckpt_dir, 'lora_a_similarity_matrices.npz'), **lora_a_sim_matrices)
np.savez(os.path.join(task_ckpt_dir, 'lora_b_similarity_matrices.npz'), **lora_b_sim_matrices)

print(f"\nAnalysis completed! Results saved to: {task_ckpt_dir}/")
print("- Individual similarity heatmaps for each key")
print("- Summary heatmaps for all keys")
print("- NPZ files containing all similarity matrices")

Task 0: Found 24 lora_A keys and 24 lora_B keys
Task 1: Found 24 lora_A keys and 24 lora_B keys
Task 2: Found 24 lora_A keys and 24 lora_B keys
Task 3: Found 24 lora_A keys and 24 lora_B keys
Task 4: Found 24 lora_A keys and 24 lora_B keys
Task 5: Found 24 lora_A keys and 24 lora_B keys
Task 6: Found 24 lora_A keys and 24 lora_B keys
Task 7: Found 24 lora_A keys and 24 lora_B keys
Task 8: Found 24 lora_A keys and 24 lora_B keys
Unique lora_A keys: ['0.0.lora_A', '0.2.lora_A', '1.0.lora_A', '1.2.lora_A', '10.0.lora_A', '10.2.lora_A', '11.0.lora_A', '11.2.lora_A', '2.0.lora_A', '2.2.lora_A', '3.0.lora_A', '3.2.lora_A', '4.0.lora_A', '4.2.lora_A', '5.0.lora_A', '5.2.lora_A', '6.0.lora_A', '6.2.lora_A', '7.0.lora_A', '7.2.lora_A', '8.0.lora_A', '8.2.lora_A', '9.0.lora_A', '9.2.lora_A']
Unique lora_B keys: ['0.0.lora_B', '0.2.lora_B', '1.0.lora_B', '1.2.lora_B', '10.0.lora_B', '10.2.lora_B', '11.0.lora_B', '11.2.lora_B', '2.0.lora_B', '2.2.lora_B', '3.0.lora_B', '3.2.lora_B', '4.0.lora_B', 